# Part 0: Dataloader and Visualizations

In [1]:
import torch
import wandb
import scipy.io

import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

from voc_dataset import VOCDataset

from PIL import Image

from utils import *

USE_WANDB = False

## Editing the Dataloader
The first part of the assignment involves editing the dataloader so that we can access bounding-box proposals as well as the ground-truth bounding boxes. The ground truth bounding box can be accessed through the VOC Dataset annotations itself. Unsupervised bounding box proposals are obtained through methods such as [Selective Search](https://ivi.fnwi.uva.nl/isis/publications/2013/UijlingsIJCV2013/UijlingsIJCV2013.pdf).

Since Selective Search is slow to run on each image, we have pre-computed the bounding box proposals. You should be able to access the `.mat` files using `scipy.io.loadmat('file.mat')`. Feel free to experiment with the data in the files to figure out the number of proposals per image, their scores, etc.

Your task is to change the dataloader to obtain the ground-truth bounding boxes, as well as the proposed bounding boxes for each image. Returning a dictionary would be convenient here. For the bounding boxes, using the relative positions is usually a better idea since they are invariant to changes in the size of the image.

In [2]:
# Load the Dataset - items at a particular index can be accesed by usual indexing notation (dataset[idx])
dataset = VOCDataset('trainval', top_n=10)

In [4]:
split = 'trainval'
selective_search_dir = os.path.join("../data/VOCdevkit/VOC2007/", 'selective_search_data')
roi_data = scipy.io.loadmat(selective_search_dir + '/voc_2007_'+ split + '.mat')



# print(roi_data.keys())
# print(roi_data['images'].shape)
# print(roi_data['images'][:,5])
# print(roi_data['boxes'][:,5][0].shape)
# print(roi_data['boxScores'][0,10].shape)
height = 200.0
box = roi_data['boxes'][0,100]
box_scores = roi_data['boxScores'][0,100]
# print(box_scores)
top_n = 10
ind = np.argpartition(box_scores, -top_n, axis=0)[-top_n:]
# print(ind)
# ind = ind[1,0]
top = box[ind]
# print(top[:,0])
# print(top[:,0][:,0])
topppy = top[:,0][:,2] 
# topppy = np.true_divide(topppy, height).reshape(10,1)
topppx = top[:,0][:,1] 
# topppx = np.true_divide(topppx, height).reshape(10,1)
# top[:,0][:,0] /= height

print(topppy.shape)

arr = np.empty((top_n, 1))
arr = np.concatenate((arr, topppy/height.reshape(10,1)), axis=1)

arr = np.concatenate((arr, topppx/height.reshape(10,1)), axis=1)
arr = arr[:,1:]
print(arr)


(10,)


AttributeError: 'float' object has no attribute 'reshape'

In [10]:
#TODO: get the image information from index 2020
idx = 2020

# input = 

ret = dataset[idx]
print(ret['rois'])

[[0.49865952 0.286      0.82841823 0.764     ]
 [0.00268097 0.862      0.46380697 1.        ]
 [0.48525469 0.378      0.86058981 0.8       ]
 [0.31099196 0.254      0.58176944 0.726     ]
 [0.47453083 0.372      0.89008043 0.726     ]
 [0.00268097 0.514      0.51474531 0.958     ]
 [0.51206434 0.372      0.68632708 0.762     ]
 [0.00268097 0.172      0.57908847 0.962     ]
 [0.51206434 0.374      0.78552279 0.762     ]
 [0.00268097 0.374      0.58981233 0.958     ]]


## Wandb Init and Logging
Initialize a Weights and Biases project, and convert the image tensor to a PIL image and plot it (check `utils.py` for helper functions).

You can use [this](https://docs.wandb.ai/library/log) as a reference for logging syntax.

In [11]:
if USE_WANDB:
    wandb.init(project="vlr2", reinit=True)

See this block as an example of plotting the ground truth box for an image.

In [24]:
class_id_to_label = dict(enumerate(dataset.CLASS_NAMES))
original_image = tensor_to_PIL(ret['image'])
gt_labels = ret['label']
print(gt_labels)
gt_labels = np.nonzero(gt_labels)
print(gt_labels[0,0])
# print(np.nonzero(gt_labels).numpy())
gt_boxes = ret['gt_boxes']
print(gt_boxes)
img = wandb.Image(original_image, boxes={
    "predictions": {
        "box_data": get_box_data(gt_labels, gt_boxes),
        "class_labels": class_id_to_label,
    },
})

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0.])
tensor(18)
[[0.236, 0.4959785522788204, 0.72, 0.5817694369973191]]


TypeError: A box's class_id must be an integer

Check the `get_box_data` function in `utils.py` and understand how it is being used. Log the image with the GT bounding box on wandb.
After, this you should be able to easily plot the top 10 bounding proposals as well.

In [9]:
rois = ret['rois']
nums = range(len(rois)) # placeholder for names of proposals

#TODO: plot top ten proposals (of bounding boxes)